In [1]:
! nvidia-smi

Wed Jun 18 00:58:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.01              Driver Version: 565.57.01      CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          On  |   00000000:52:00.0 Off |                    0 |
| N/A   33C    P0             46W /  270W |       1MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
import argparse
import os
import time
from typing import Dict, Tuple, Union, Optional, Callable, List, Any
from torch.utils.data import Dataset, DataLoader, Subset
import numpy as np
import torch
import torch.distributed as dist
import transformers
import yaml
from datasets import (
    Dataset,
    load_dataset,
    DatasetDict,
    IterableDatasetDict,
    IterableDataset,
)
from datasets import Dataset as HFDataset, DatasetDict
from sklearn.metrics import f1_score, matthews_corrcoef
from sklearn.model_selection import KFold
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    PreTrainedTokenizer,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
    DataCollatorWithPadding,
    PreTrainedModel,
    AutoConfig,
)



/home/wenduoc/mambaforge/envs/dnalongbench/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/wenduoc/mambaforge/envs/dnalongbench/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2025-06-18 00:58:34.178990: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-18 00:58:34.192934: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register fact

In [3]:
import tensorflow as tf
import torch
import numpy as np
import os
import json
from natsort import natsorted
import glob

In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
tf.config.set_visible_devices([], 'GPU')

In [6]:
!nvidia-smi

Wed Jun 18 00:58:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.01              Driver Version: 565.57.01      CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          On  |   00000000:52:00.0 Off |                    0 |
| N/A   33C    P0             46W /  270W |       4MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [8]:
root = '/work/magroup/shared/DNA_LLM/DNALongBench/'

In [9]:
cell_type="IMR90"
batch_size=4

In [10]:
train_data_path = root + 'contact_map_prediction/targets/train-*.tfr'
valid_data_path = root + 'contact_map_prediction/targets/valid-*.tfr'
test_data_path = root + 'contact_map_prediction/targets/test-*.tfr'
SEQUENCE_LENGTH = 1048576
TARGET_LENGTH = 99681

In [11]:
class AkitaDataset(torch.utils.data.IterableDataset):
    def __init__(self, tfr_pattern, cell_type):
        super(AkitaDataset).__init__()
        self.dataset = self.read_tfr(tfr_pattern)
        self.cell_type = cell_type
        target_ind_dict = {'HFF': 0, 'H1hESC': 1, 'GM12878': 2, 'IMR90': 3, 'HCT116': 4}
        self.target_ind = target_ind_dict[self.cell_type]

    def file_to_records(self, filename):
        return tf.data.TFRecordDataset(filename, compression_type='ZLIB')

    def parse_proto(self, example_protos):
        features = {
            'sequence': tf.io.FixedLenFeature([], tf.string),
            'target': tf.io.FixedLenFeature([], tf.string)
          }
        parsed_features = tf.io.parse_example(example_protos, features=features)
        seq = tf.io.decode_raw(parsed_features['sequence'], tf.uint8)
        targets = tf.io.decode_raw(parsed_features['target'], tf.float16)
        return seq, targets

    def read_tfr(self, tfr_pattern):
        tfr_files = natsorted(glob.glob(tfr_pattern))
        if tfr_files:
            dataset = tf.data.Dataset.list_files(tf.constant(tfr_files), shuffle=False)
        else:
            print('Cannot order TFRecords %s' % tfr_pattern, file=sys.stderr)
            dataset = tf.data.Dataset.list_files(tfr_pattern)
        dataset = dataset.flat_map(self.file_to_records)
        dataset = dataset.map(self.parse_proto)
        dataset = dataset.batch(1)
        return dataset

    def __iter__(self):
        num = 200
        for seq_raw, targets_raw in self.dataset:
            # print(seq_raw.shape, targets_raw.shape)\n",
            seq = seq_raw.cpu().numpy().reshape(-1, 4).astype('int8')
            targets = targets_raw.cpu().numpy().reshape(TARGET_LENGTH, -1).astype('float16')
            # yield {"sequence": seq, "target": targets[:, self.target_ind]}
            # seq = seq[-372736: -65536, :]
            seq = seq[-475136: -65536, :]
            # seq = np.argmax(seq, axis=-1)
            targets = targets[:, self.target_ind]
            # targets = targets[-11026:]
            targets = targets[-19701:]
            scores = np.eye(num)
            index = 0
            for i in range(num):
                if i < num - 1:
                    scores[i][i + 1] = 1
                for j in range(i + 2, num):
                    scores[i][j] = targets[index]
                    index += 1
            for i in range(num):
                for j in range(i - 1):
                    scores[i][j] = scores[j][i]
            scores = torch.FloatTensor(scores).reshape(-1)
            yield (seq, scores)

In [12]:


train_dataset = AkitaDataset(train_data_path, cell_type = cell_type)
valid_dataset = AkitaDataset(valid_data_path, cell_type = cell_type)
test_dataset = AkitaDataset(test_data_path, cell_type = cell_type)


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=0)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=0)

In [13]:
for batch in train_loader: 
        x, y = batch
        print('x:',x.size())
        print('y:',y.size())
        break


Instructions for updating:
Use tf.identity with explicit device placement instead.
x: torch.Size([4, 409600, 4])
y: torch.Size([4, 40000])


2025-06-18 00:58:37.562860: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


In [14]:
!nvidia-smi

Wed Jun 18 00:58:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.01              Driver Version: 565.57.01      CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          On  |   00000000:52:00.0 Off |                    0 |
| N/A   33C    P0             65W /  270W |       4MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [15]:

# Set logging level for transformers
transformers.logging.set_verbosity_info()

# Define optimization direction for each metric (whether higher or lower is better)
METRICS_DIRECTION: Dict[str, str] = {
    "accuracy": "max",
    "f1_score": "max",
    "mcc": "max",
    "f1_max": "max",
    "auprc_micro": "max",
    "mse": "min",
    "mae": "min",
    "r2": "max",
    "pearson": "max",
}


def is_main_process() -> bool:
    """
    Check if current process is the main process (rank 0) in distributed training.

    Returns:
        bool: True if this is the main process, False otherwise
    """
    if dist.is_initialized():
        return dist.get_rank() == 0
    return True


def dist_print(*args, **kwargs) -> None:
    """
    Print only from the main process (rank 0) in distributed training.
    Prevents duplicate outputs in multi-GPU settings.

    Args:
        *args: Arguments to pass to print function
        **kwargs: Keyword arguments to pass to print function
    """
    if is_main_process():
        print(*args, **kwargs)


In [16]:
def setup_tokenizer(
    model_name: str, padding_and_truncation_side: str
) -> PreTrainedTokenizer:
    """
    Load and configure tokenizer for sequence understanding.

    Args:
        model_name: Name or path of the HuggingFace model
        padding_and_truncation_side: Side for padding and truncation (left or right)

    Returns:
        PreTrainedTokenizer: Configured tokenizer for the model
    """
    dist_print(f"🔤 Loading tokenizer from: {model_name}")
    start_time = time.time()

    # Load tokenizer with trust_remote_code to support custom models
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

    # Configure padding and truncation settings
    tokenizer.padding_side = padding_and_truncation_side
    tokenizer.truncation_side = padding_and_truncation_side

    # Set pad_token to eos_token if not defined
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    dist_print(
        f"⏱️ Tokenizer loading completed in {time.time() - start_time:.2f} seconds"
    )

    return tokenizer

In [17]:
tokenizer = AutoTokenizer.from_pretrained("GenerTeam/GENERator-eukaryote-1.2b-base", trust_remote_code=True) # "GenerTeam/GENERator-eukaryote-3b-base"

loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/wenduoc/.cache/huggingface/hub/models--GenerTeam--GENERator-eukaryote-1.2b-base/snapshots/3be4abf390afbb7f4d8ccb3370f599338523f1cd/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/wenduoc/.cache/huggingface/hub/models--GenerTeam--GENERator-eukaryote-1.2b-base/snapshots/3be4abf390afbb7f4d8ccb3370f599338523f1cd/tokenizer_config.json
loading file tokenizer.json from cache at None
loading file chat_template.jinja from cache at None


In [18]:
tokenizer

DNAKmerTokenizer(name_or_path='GenerTeam/GENERator-eukaryote-1.2b-base', vocab_size=4128, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<oov>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("<oov>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [19]:
def collate_fn(batch, tokenizer, max_length=409600):
    """
    Custom collate function for DNA data that converts one-hot encoded sequences to raw sequences
    and tokenizes them.
    
    Args:
        batch: List of tuples where each tuple is (x, y)
               x is one-hot encoded DNA sequence of shape (seq_len, 4)
               y is gene expression data of shape (10, seq_len)
        tokenizer: The GENERator tokenizer
        max_length: Maximum sequence length for tokenization
    
    Returns:
        Dictionary with tokenized inputs and original gene expression data
    """
    # Separate x and y from the batch
    x_batch, y_batch = zip(*batch)
    
    # Convert one-hot encoded sequences to raw sequences
    raw_sequences = []
    nucleotides = ['A', 'C', 'G', 'T']
    for one_hot_seq in x_batch:
        # Ensure one_hot_seq is a PyTorch tensor
        if not isinstance(one_hot_seq, torch.Tensor):
            one_hot_seq = torch.tensor(one_hot_seq)
        
        # Get indices of 1s in one-hot encoding (argmax along axis 1)
        indices = torch.argmax(one_hot_seq, dim=1).cpu().numpy()
        
        # Convert indices to nucleotides
        raw_seq = ''.join([nucleotides[idx] for idx in indices])
        raw_sequences.append(raw_seq)
    
    # Tokenize the raw sequences
    tokenizer.padding_side = "right"
    inputs = tokenizer(
        raw_sequences,
        add_special_tokens=True,
        return_tensors="pt",
        padding=False,
        truncation=True,
        # max_length=max_length
    )

    
    # Convert y arrays to tensors and stack them
    y_tensors = []
    for y in y_batch:
        if not isinstance(y, torch.Tensor):
            y = torch.tensor(y, dtype=torch.float32)
        y_tensors.append(y)
    
    y_stacked = torch.stack(y_tensors)
    
    # Return tokenized inputs and original y
    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "y": y_stacked
    }

In [20]:
train_loader2 = DataLoader(
        train_loader.dataset,
        batch_size=1,
        collate_fn=lambda b: collate_fn(b, tokenizer, max_length=409600)
    )


In [21]:
for batch in train_loader2: 
        print(batch)
        break


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': tensor([[   1,   47, 2167,  ...,  868,    0,    2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]]), 'y': tensor([[1.0000, 1.0000, 0.0611,  ..., 0.0790, 0.0000, 1.0000]])}


In [22]:
batch['input_ids'].shape, batch['attention_mask'].shape, batch['y'].shape

(torch.Size([1, 68269]), torch.Size([1, 68269]), torch.Size([1, 40000]))

In [23]:
!nvidia-smi

Wed Jun 18 00:58:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.01              Driver Version: 565.57.01      CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          On  |   00000000:52:00.0 Off |                    0 |
| N/A   33C    P0             65W /  270W |       4MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [24]:
import torch
import torch.nn as nn
from transformers import AutoModel

class LongSequenceRegressionModel(nn.Module):
    def __init__(
        self,
        base_model_name: str,
        num_labels: int = 2,
        max_subsequence_length: int = 9375,
        num_subsequences: int = 8,
        gradient_checkpointing: bool = True
    ):
        super().__init__()
        self.base_model = AutoModel.from_pretrained(base_model_name, trust_remote_code=True)
        if gradient_checkpointing:
            self.base_model.gradient_checkpointing_enable()

        hidden_size = self.base_model.config.hidden_size
        self.regression_head = nn.Linear(num_subsequences * hidden_size, num_labels, bias=False)

        self.max_subsequence_length = max_subsequence_length
        self.num_subsequences      = num_subsequences

    def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor = None):
        """
        input_ids:        [B, L]
        attention_mask:   [B, L] (optional — if None, we treat it as all 1’s)
        """
        # if no mask provided, assume everything is valid
        if attention_mask is None:
            attention_mask = torch.ones_like(input_ids)

        hidden_states = []
        for i in range(self.num_subsequences):
            start_idx = i * self.max_subsequence_length
            end_idx   = (i + 1) * self.max_subsequence_length

            sub_input_ids      = input_ids[:, start_idx:end_idx]
            sub_attention_mask = attention_mask[:, start_idx:end_idx]

            outputs = self.base_model(
                input_ids=sub_input_ids,
                attention_mask=sub_attention_mask
            )
            # take the CLS token embedding
            cls_embedding = outputs.last_hidden_state[:, -1, :]
            hidden_states.append(cls_embedding)

        combined = torch.cat(hidden_states, dim=-1)   # [B, num_subseq * H]
        logits   = self.regression_head(combined)     # [B, num_labels]
        return {"logits": logits}


In [25]:
model = LongSequenceRegressionModel(
    base_model_name="GenerTeam/GENERator-eukaryote-1.2b-base",
    num_labels=40000,
    max_subsequence_length=8534,
    num_subsequences=8
)

loading configuration file config.json from cache at /home/wenduoc/.cache/huggingface/hub/models--GenerTeam--GENERator-eukaryote-1.2b-base/snapshots/3be4abf390afbb7f4d8ccb3370f599338523f1cd/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 16384,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 26,
  "num_key_value_heads": 4,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 500000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3",
  "use_cache": true,
  "vocab_size": 4128
}

loading weights file model.safetensors from cache at /home/wenduoc/.cache/huggingface/hub/models-

In [26]:
model=model.to(torch.bfloat16).to(device)

In [27]:
model

LongSequenceRegressionModel(
  (base_model): LlamaModel(
    (embed_tokens): Embedding(4128, 2048)
    (layers): ModuleList(
      (0-25): 26 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
   

In [28]:
for batch in train_loader2: 
        print(batch)
        break


{'input_ids': tensor([[   1,   47, 2167,  ...,  868,    0,    2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]]), 'y': tensor([[1.0000, 1.0000, 0.0611,  ..., 0.0790, 0.0000, 1.0000]])}


In [29]:
batch['input_ids'].shape

torch.Size([1, 68269])

In [32]:
from tqdm import tqdm
import torch
import torch.nn as nn
import numpy as np
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import os
from typing import Dict, Any, Optional
from torch.utils.data import DataLoader
from transformers import PreTrainedModel, PreTrainedTokenizer


def evaluate_model_custom(
    model: PreTrainedModel,
    data_loader: DataLoader,
    device: str,
    criterion: nn.Module
) -> Dict[str, float]:
    """
    Evaluate regression model on a dataset.
    
    Args:
        model: Model to evaluate
        data_loader: DataLoader for evaluation
        device: Device to run evaluation on
        criterion: Loss function (MSE)
    
    Returns:
        Dictionary of evaluation metrics (MSE, MAE, R², PCC)
    """
    model.eval()
    total_loss = 0.0
    all_predictions = []
    all_labels = []
    num_batches = 0
    
    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['y'].to(device)

            # input_ids = input_ids.to(model.dtype)
            # attention_mask = attention_mask.to(model.dtype)
            
            outputs = model(input_ids=input_ids,attention_mask=attention_mask)
            logits = outputs['logits']
            
            # Process labels for regression
            if labels.dim() > 1:
                labels = labels.view(-1).float()
            
            # Calculate MSE loss
            loss = criterion(logits.view(-1), labels)
            
            # Get predictions (no activation function needed for regression)
            predictions = logits.float().cpu().numpy()
     
            total_loss += loss.item()
            
            # Store for metrics
            all_predictions.extend(predictions.flatten())
            all_labels.extend(labels.cpu().numpy().flatten())
                
            num_batches += 1
    
    # Compute average loss
    avg_loss = total_loss / num_batches
    
    # Convert lists to numpy arrays
    all_predictions = np.array(all_predictions)
    all_labels = np.array(all_labels)
    
    # Mean Squared Error
    mse = mean_squared_error(all_labels, all_predictions)
    
    # Mean Absolute Error
    mae = mean_absolute_error(all_labels, all_predictions)
    
    # R-squared score
    r2 = r2_score(all_labels, all_predictions)
    
    # Pearson Correlation Coefficient
    pcc, p_value = pearsonr(all_labels, all_predictions)
    
    # Root Mean Squared Error
    rmse = np.sqrt(mse)
    
    return {
        'loss': avg_loss,
        'mse': mse,
        'mae': mae,
        'rmse': rmse,
        'r2': r2,
        'pcc': pcc,
        'pcc_p_value': p_value,
        'num_samples': len(all_predictions)
    }


def train_model_custom(
    model: PreTrainedModel,
    tokenizer: PreTrainedTokenizer,
    train_loader: DataLoader,
    val_loader: DataLoader,
    test_loader: Optional[DataLoader] = None,
    num_epochs: int = 10,
    learning_rate: float = 1e-4,
    weight_decay: float = 0.01,
    warmup_steps: int = 0,
    max_grad_norm: float = 1.0,
    save_dir: str = "/work/magroup/wenduoc/DNALongBench/experiments/GENERator/results/CMP/GM12878",
    save_steps: int = 1000,
    early_stopping_patience: int = 5,
    device: str = "cuda",
    use_wandb: bool = False,
    gradient_accumulation_steps: int = 1,
    max_length: int = 75002,
) -> Dict[str, Any]:
    """
    Custom training function for DNA sequence models with gene expression prediction (regression).
    Modified to evaluate after each epoch and save model based on lowest validation loss (MSE).
    """
    model = model.to(device)
    model.train()

    train_loader_custom = DataLoader(
        train_loader.dataset,
        batch_size=1,
        collate_fn=lambda b: collate_fn(b, tokenizer)
    )

    val_loader_custom = DataLoader(
        val_loader.dataset,
        batch_size=1,
        collate_fn=lambda b: collate_fn(b, tokenizer)
    )

    if test_loader is not None:
        test_loader_custom = DataLoader(
            test_loader.dataset,
            batch_size=1,
            collate_fn=lambda b: collate_fn(b, tokenizer)
        )

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=learning_rate,
        weight_decay=weight_decay
    )

    scheduler = torch.optim.lr_scheduler.LinearLR(
        optimizer,
        start_factor=0.1,
        end_factor=1.0,
        total_iters=warmup_steps
    )

    # Use MSE loss for regression
    criterion = nn.MSELoss()

    best_val_loss = float('inf')
    best_val_pcc = -1.0  # Track best PCC as well
    patience_counter = 0
    global_step = 0
    training_history = {
        'train_loss': [],
        'val_loss': [],
        'val_pcc': [],
        'val_r2': [],
        'learning_rates': []
    }

    print(f"🚀 Starting regression training for {num_epochs} epochs...")
    print(f"🔧 Gradient accumulation steps: {gradient_accumulation_steps}")
    print(f"📊 Evaluation will occur after each epoch")
    print(f"💾 Model will be saved based on lowest validation MSE loss")

    start_time = time.time()

    for epoch in range(num_epochs):
        print(f"\n{'='*50}")
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"{'='*50}")

        model.train()
        epoch_train_loss = 0.0
        train_steps = 0

        progress_bar = tqdm(train_loader_custom, desc=f"Epoch {epoch + 1}")

        for step, batch in enumerate(progress_bar):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['y'].to(device)

            # input_ids = input_ids.to(model.dtype)
            # attention_mask = attention_mask.to(model.dtype)

            with torch.cuda.amp.autocast():
                outputs = model(input_ids=input_ids,attention_mask=attention_mask)
                logits = outputs['logits']

                if labels.dim() > 1:
                    labels = labels.view(-1).float()

                # MSE loss for regression
                loss = criterion(logits.view(-1), labels)
                loss = loss / gradient_accumulation_steps

            loss.backward()

            if (step + 1) % gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                global_step += 1

            epoch_train_loss += loss.item() * gradient_accumulation_steps
            train_steps += 1

            progress_bar.set_postfix({
                'mse_loss': f"{(loss.item() * gradient_accumulation_steps):.4f}",
                'lr': f"{scheduler.get_last_lr()[0]:.2e}"
            })

            # if global_step % save_steps == 0:
            #     checkpoint_path = os.path.join(save_dir, f'checkpoint_step_{global_step}.pt')
            #     torch.save({
            #         'epoch': epoch,
            #         'model_state_dict': model.state_dict(),
            #         'optimizer_state_dict': optimizer.state_dict(),
            #         'scheduler_state_dict': scheduler.state_dict(),
            #         'global_step': global_step
            #     }, checkpoint_path)
            #     print(f"💾 Intermediate checkpoint saved at step {global_step}")
         

        avg_train_loss = epoch_train_loss / train_steps
        training_history['train_loss'].append(avg_train_loss)
        training_history['learning_rates'].append(scheduler.get_last_lr()[0])

        print(f"\n🔄 Evaluating after epoch {epoch + 1}...")
        val_metrics = evaluate_model_custom(model, val_loader_custom, device, criterion)
        training_history['val_loss'].append(val_metrics['loss'])
        training_history['val_pcc'].append(val_metrics['pcc'])
        training_history['val_r2'].append(val_metrics['r2'])

        print(f"\n📈 Epoch {epoch + 1} Summary:")
        print(f"  Train MSE Loss: {avg_train_loss:.4f}")
        print(f"  Val MSE Loss: {val_metrics['loss']:.4f}")
        print(f"  Val PCC: {val_metrics['pcc']:.4f}")
        print(f"  Val R²: {val_metrics['r2']:.4f}")
        print(f"  Val RMSE: {val_metrics['rmse']:.4f}")
        print(f"  Learning Rate: {scheduler.get_last_lr()[0]:.2e}")

        # Save model based on lowest validation loss
        if val_metrics['loss'] < best_val_loss:
            best_val_loss = val_metrics['loss']
            best_val_pcc = val_metrics['pcc']
            patience_counter = 0

            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'best_val_loss': best_val_loss,
                'best_val_pcc': best_val_pcc,
                'global_step': global_step
            }, os.path.join(save_dir, 'best_model.pt'))

            print(f"💾 New best model saved! Val MSE: {best_val_loss:.4f}, Val PCC: {best_val_pcc:.4f}")
        else:
            patience_counter += 1
            print(f"⏳ No improvement in MSE loss. Patience: {patience_counter}/{early_stopping_patience}")

        epoch_checkpoint_path = os.path.join(save_dir, f'model_epoch_{epoch + 1}.pt')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'val_loss': val_metrics['loss'],
            'val_pcc': val_metrics['pcc'],
            'global_step': global_step
        }, epoch_checkpoint_path)
        print(f"💾 Epoch {epoch + 1} checkpoint saved")

        if patience_counter >= early_stopping_patience:
            print(f"🛑 Early stopping triggered after {patience_counter} epochs without MSE loss improvement")
            break

        model.train()

    total_time = time.time() - start_time
    print(f"\n✅ Training completed in {total_time/60:.2f} minutes")
    print(f"🏆 Best validation MSE loss achieved: {best_val_loss:.4f}")
    print(f"🏆 Best validation PCC achieved: {best_val_pcc:.4f}")

    final_metrics = {
        'training_history': training_history, 
        'best_val_loss': best_val_loss,
        'best_val_pcc': best_val_pcc
    }
    
    if test_loader is not None:
        print("\n🧪 Evaluating on test set...")
        test_metrics = evaluate_model_custom(model, test_loader_custom, device, criterion)
        final_metrics['test_metrics'] = test_metrics

        print("📊 Final Test Metrics:")
        print(f"  MSE: {test_metrics['mse']:.4f}")
        print(f"  RMSE: {test_metrics['rmse']:.4f}")
        print(f"  MAE: {test_metrics['mae']:.4f}")
        print(f"  R²: {test_metrics['r2']:.4f}")
        print(f"  PCC: {test_metrics['pcc']:.4f}")
        print(f"  PCC p-value: {test_metrics['pcc_p_value']:.6f}")

    return final_metrics

In [ ]:


# Train the model
training_results = train_model_custom(
    model=model,
    tokenizer=tokenizer,
    train_loader=train_loader,
    val_loader=valid_loader,
    test_loader=test_loader,
    num_epochs=5,
    learning_rate=1e-4,
    # batch_size=1,  # Start small due to memory constraints
    max_length=68269,
    device=device,
    use_wandb=False,  
    gradient_accumulation_steps=8,  # Effective batch size = 1 * 8 = 8
)



🚀 Starting regression training for 5 epochs...
🔧 Gradient accumulation steps: 8
📊 Evaluation will occur after each epoch
💾 Model will be saved based on lowest validation MSE loss

Epoch 1/5


Epoch 1: 0it [00:00, ?it/s]/tmp/ipykernel_1116178/1156599230.py:201: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1: 1754it [3:52:08,  7.97s/it, mse_loss=0.1937, lr=1.00e-05]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

Epoch 1: 7008it [15:24:12,  7.90s/it, mse_loss=0.1025, lr=1.00e-05]2025-06-18 16:35:07.075243: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
Epoch 1: 7008it [15:24:12,  7.91s/it, mse_loss=0.1025, lr=1.00e-05]



🔄 Evaluating after epoch 1...


Evaluating: 419it [14:19,  2.06s/it]2025-06-18 16:49:26.563260: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
Evaluating: 419it [14:19,  2.05s/it]



📈 Epoch 1 Summary:
  Train MSE Loss: 0.6878
  Val MSE Loss: 0.1310
  Val PCC: 0.2006
  Val R²: -0.0528
  Val RMSE: 0.3620
  Learning Rate: 1.00e-05
💾 New best model saved! Val MSE: 0.1310, Val PCC: 0.2006
💾 Epoch 1 checkpoint saved

Epoch 2/5


Epoch 2: 0it [00:00, ?it/s]/tmp/ipykernel_1116178/1156599230.py:201: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 2: 417it [54:47,  7.90s/it, mse_loss=0.1516, lr=1.00e-05]

In [ ]:
# Final evaluation on test set if provided
final_metrics = {}

test_loader_custom = DataLoader(
            test_loader.dataset,
            batch_size=1,
            collate_fn=lambda b: collate_fn(b, tokenizer)
        )

criterion = nn.MSELoss()

print("\n🧪 Evaluating on test set...")
test_metrics = evaluate_model_custom(model, test_loader_custom, device, criterion)
final_metrics['test_metrics'] = test_metrics

print("📊 Final Test Metrics:")
for key, value in test_metrics.items():
    print(f"  {key}: {value:.4f}")

print(final_metrics)